In [0]:
import pandas as pd
from PIL import Image
import numpy as np
import cv2
import os
import glob
from google.colab import drive
import bottleneck as bn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler 
from sklearn.preprocessing import scale
from sklearn.metrics import classification_report
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from scipy import ndimage, misc

In [0]:
drive.mount('/content/gdrive')
data=[]
for i in range(1,41):
  for j in range(1,11):
    path='/content/gdrive/My Drive/Colab Notebooks/data/s'+str(i)+'/'
    img=Image.open(path+str(j)+'.pgm')
    temp=np.array(img).reshape(10304)
    data.append(temp)

D = np.array([x for x in data])
D = D.reshape((400, 10304))

In [0]:
labels = []
for i in range(1, 41):
  labels.append(i * np.ones((10, 1)))
  
y = np.stack(labels, axis=0).reshape(400, 1)

X_train = D[::2]
y_train = y[::2]
X_test = D[1::2]
y_test = y[1::2]

**LDA** **Calculation**

In [0]:
means=np.zeros([10304,40])
for i in range(40):
  temp=np.mean(X_train[i*5:(i*5)+5],axis=0)
  means[:,i]=temp

In [0]:
overallMean=(np.mean(X_train,axis=0))
Sb=np.zeros([10304,10304])
for i in range(40):
  Sb += 5*np.outer((means[:,i]-overallMean),np.transpose(means[:,i]-overallMean))
Sb=np.array(Sb)

In [0]:
Z=np.zeros([200,10304])
for i in range(40):
  Z[5*i:(5*i)+5,:]=X_train[5*i:(5*i)+5,:]-means[:,i-1]

In [0]:
S=np.zeros([10304,10304])
for i in range(0,40):
  temp=np.matmul(np.transpose(Z[5*i:(5*i)+5,:]),Z[5*i:(5*i)+5,:])
  S=S+temp

In [0]:
SInverse=np.linalg.pinv(S,hermitian=True)

In [0]:
SInverseB=np.matmul(SInverse,Sb)

In [0]:
eigenValues,eigenVectors = np.linalg.eig(SInverseB)

In [0]:
idx = eigenValues.argsort()[::-1]
eigenValues =eigenValues[idx]
eigenVectors =eigenVectors[:,idx]
U=np.zeros([10304,39])
U=eigenVectors[:,:39]

Projection on Data

In [0]:
projection_train = X_train.dot((U))
projection_test=X_test.dot((U))

projected_train = np.array(projection_train)
projected_test = np.array(projection_test)

projected_train = projected_train.real
projected_test = projected_test.real
print (projected_train.shape)
print(y_train.shape)
neighbours=[1,3,5,7]
accuracy=[]
for neighbor in neighbours:
  KNN = KNeighborsClassifier(n_neighbors=neighbor)
  KNN.fit(projected_train, y_train) 
  prediction = KNN.predict(projected_test)
  print(neighbor , ' neighbour total accuracy:', accuracy_score( y_test, prediction), '\n\n')
  accuracy.append(accuracy_score( y_test, prediction))
print(np.array(accuracy))

In [0]:
plt.plot(neighbours,accuracy)
plt.title("Accuracy to Number of Neighbors")
plt.xlabel("K Neighbors")
plt.ylabel("Accuracy")
plt.show()